In [1]:
###-----------------
### Import Libraries
###-----------------

import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import KFold,cross_val_score
from sklearn.metrics import log_loss,accuracy_score,mean_squared_error
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings('ignore')

In [2]:
img=pd.read_csv('Image_Segmention.csv')


img.head()

,Class,region.centroid.col,region.centroid.row,region.pixel.count,short.line.density.5,short.line.density.2,vedge.mean,vegde.sd,hedge.mean,hedge.sd,intensity.mean,rawred.mean,rawblue.mean,rawgreen.mean,exred.mean,exblue.mean,exgreen.mean,value.mean,saturation.mean,hue-mean
0,BRICKFACE,188,133,9,0.0,0.0,0.333333,0.266667,0.500000,0.077778,6.666666,8.333334,7.777778,3.888889,5.000000,3.333333,-8.333333,8.444445,0.538580,-0.924817
1,BRICKFACE,105,139,9,0.0,0.0,0.277778,0.107407,0.833333,0.522222,6.111111,7.555555,7.222222,3.555556,4.333334,3.333333,-7.666666,7.555555,0.532628,-0.965946
2,BRICKFACE,34,137,9,0.0,0.0,0.500000,0.166667,1.111111,0.474074,5.851852,7.777778,6.444445,3.333333,5.777778,1.777778,-7.555555,7.777778,0.573633,-0.744272
3,BRICKFACE,39,111,9,0.0,0.0,0.722222,0.374074,0.888889,0.429629,6.037037,7.000000,7.666666,3.444444,2.888889,4.888889,-7.777778,7.888889,0.562919,-1.175773
4,BRICKFACE,16,128,9,0.0,0.0,0.500000,0.077778,0.666667,0.311111,5.555555,6.888889,6.666666,3.111111,4.000000,3.333333,-7.333334,7.111111,0.561508,-0.985811


In [3]:
lbl=LabelEncoder()
img['Class']=lbl.fit_transform(img['Class'])
print(lbl.classes_)

['BRICKFACE' 'CEMENT' 'FOLIAGE' 'GRASS' 'PATH' 'SKY' 'WINDOW']


In [4]:
x=img.drop('Class',axis=1)
y=img['Class']

In [5]:
x_train,x_test,y_train,y_test=train_test_split(x,y,
                                               test_size=0.3,
                                               random_state=23,
                                               stratify=y)

In [6]:
lr=LogisticRegression()    #multi_class= multinomial
lr.fit(x_train,y_train)

LogisticRegression()

In [7]:
y_pred_proba=lr.predict_proba(x_test)
print(log_loss(y_test,y_pred_proba))

y_pred=lr.predict(x_test)
print(accuracy_score(y_test,y_pred))

0.27163401168459117
0.9206349206349206


# ----------------------------------------------------------------------------------------------------------

In [8]:
#Grid search cv
kfold=StratifiedKFold(n_splits=5,shuffle=True,random_state=23)
mse=mean_squared_error

In [9]:
params={'penalty':["l1","l2","elastic",None],
       'solver':['lbfgs','liblinear','newton-cg','newton-cholesky','sag','saga'],
        'multi_class':['ovr', 'multinomial']}
                                            #multi_class{‘auto’, ‘ovr’, ‘multinomial’}, default=’auto’

gcv=GridSearchCV(lr,param_grid=params,cv=kfold)
gcv.fit(x,y)

print(gcv.best_params_)
print(gcv.best_score_)


print(mse(y_test,y_pred))

{'multi_class': 'ovr', 'penalty': 'l2', 'solver': 'newton-cg'}
0.9044134727061557
1.4444444444444444


In [10]:
gcv=GridSearchCV(lr,param_grid=params,cv=kfold,scoring='neg_log_loss')
gcv.fit(x,y)

print(gcv.best_params_)
print(gcv.best_score_)

{'multi_class': 'multinomial', 'penalty': 'l2', 'solver': 'newton-cg'}
-0.5465525609071172


In [11]:
bm=gcv.best_estimator_

print(bm.coef_)
print(bm.fit_intercept)

[[-3.02403569e-02 -1.29097151e-01  1.05406506e-03 -1.64282484e-02
  -4.54631921e-05 -2.65706121e-01 -1.25123165e-01  3.68222555e-02
  -9.53361863e-02 -7.55518396e-02  2.68787652e-01 -3.37518384e-02
  -4.61691898e-01  1.03301938e+00  1.25400437e-01 -1.15841949e+00
  -5.48293635e-02 -2.48404226e-02  7.18655748e-02]
 [-5.85826462e-03 -5.97605861e-02 -1.98316736e-04  1.14007534e-01
   7.06925154e-02 -8.39163922e-02  1.49268512e-01  2.94176341e-01
  -7.06496922e-02  1.37638236e-01  2.77752016e-01  2.32265216e-01
  -9.71032078e-02  4.20339034e-01  2.83883328e-01 -7.04223218e-01
  -5.77254182e-01  2.11481520e-02 -1.78948045e-01]
 [-8.98890654e-03 -1.69606871e-01 -8.21690572e-04 -1.92327700e-02
   1.27682667e-03  2.86385357e-01  6.23072934e-02 -5.11576619e-02
   4.79211953e-02 -2.75448329e-01 -5.34992721e-01 -3.47056044e-01
   5.57027295e-02 -7.78633341e-01 -2.14820623e-01  9.93454056e-01
   7.11871289e-01  7.45091432e-01 -5.94164649e-01]
 [ 1.74678052e-02  2.17556125e-01 -1.55326435e-03  2.30

In [12]:
#Inferencing
tst_img=pd.read_csv("tst_img.csv")

y_pred=bm.predict(tst_img)

print(lbl.inverse_transform(y_pred))

['SKY' 'PATH' 'GRASS' 'GRASS']
